In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, GlobalAveragePooling2D
from PIL import Image
import numpy as np

import matplotlib.pyplot as plt

from getData import *
from vggFace import vggFace

model = vggFace()


Using TensorFlow backend.


Constructing model
loading weights


weights loaded


In [5]:
# CLEAR TEST/TRAIN DATA
tr_imgs = 0
test_imgs = 0

In [2]:

tr_imgs, tr_labels, tr_ids = getTrainingDataReplay_w_id()
test_imgs, test_labels,test_ids = getTestDataReplay_w_id()


# tr_imgs, tr_labels = getTrainingDataOuluP1()
# test_imgs, test_labels = getTestDataOuluP1()

print(tr_imgs.shape, tr_labels.shape)
print(test_imgs.shape, test_labels.shape)

(2400, 224, 224, 3) (2400,)
(3200, 224, 224, 3) (3200,)


In [3]:
# test_imgs = test_imgs[0:20,:,:,:]
# print(test_imgs.shape)
# print(tr_ids.shape)
dummy_test_ids = np.random.rand(3200,15)

In [ ]:
# test_imgs, test_labels = getTestDataOuluP1()
# test_imgs, test_labels = getTestDataReplay()
# print(test_imgs.shape, test_labels.shape)

In [4]:
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.metrics import binary_accuracy
# pad_model = Model(inputs=model.layers[0].input, outputs=model.get_layer('last_feature_layer').output)
x = GlobalAveragePooling2D()(model.get_layer('last_feature_layer').output)
x = Dense(100,activation='relu',name="pad2")(x)
x = Dense(80,activation='relu', name="pad1")(x)
x = Dense(1,activation='tanh', name='pad')(x)

y = Flatten()(model.get_layer("fr_classif5").output)
y = Dense(15,name="fr_classif")(y)
y = Activation('softmax', name='fr')(y)

pad_model = Model(inputs=model.layers[0].input, outputs=[x,y])


# for layer in pad_model.layers:
#     print(layer.name)

# pad_model.get_layer("fr_classif1").trainable = False
# pad_model.get_layer("fr_classif2").trainable = False
# pad_model.get_layer("fr_classif3").trainable = False
# pad_model.get_layer("fr_classif4").trainable = False
# pad_model.get_layer("fr_classif5").trainable = False

for layer in pad_model.layers:
    layer.trainable = False

pad_model.summary()

# model.compile(optimizer='adam', loss='mse', metrics=['accuracy',binary_accuracy])
losses = {'pad' : 'mse', 'fr' : 'categorical_crossentropy'}
# lossWeights = {"pad": 1.0, "fr": 1.0}
# loss_weights=lossWeights
pad_model.compile(optimizer='adam', loss=losses, metrics=['accuracy','categorical_accuracy'])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
zero_padding2d_1_input (InputLa (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 226, 226, 3)  0           zero_padding2d_1_input[0][0]     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 64) 1792        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 226, 226, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (

In [5]:
pad_model.get_layer("pad2").trainable = True
pad_model.get_layer("pad1").trainable = True
pad_model.get_layer("pad").trainable = True
pad_model.get_layer("fr_classif").trainable = True
pad_model.get_layer("fr").trainable = True
pad_model.compile(optimizer='adam', loss=losses, metrics=['accuracy','categorical_accuracy'])

In [6]:
history = pad_model.fit(tr_imgs, {"pad":tr_labels, "fr":tr_ids}, batch_size=16, epochs=2)
print(history.history)
pad_model.save("padModel4_1_Replay_28122018.h5")

Epoch 1/2


  16/2400 [..............................] - ETA: 3:34 - loss: 8.6293 - pad_loss: 2.2199 - fr_loss: 6.4093 - pad_acc: 0.3125 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.0625 - fr_categorical_accuracy: 0.0625

  32/2400 [..............................] - ETA: 1:57 - loss: 7.5476 - pad_loss: 1.5197 - fr_loss: 6.0279 - pad_acc: 0.4375 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.0938 - fr_categorical_accuracy: 0.0938

  48/2400 [..............................] - ETA: 1:25 - loss: 7.6338 - pad_loss: 1.6976 - fr_loss: 5.9362 - pad_acc: 0.3958 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.1042 - fr_categorical_accuracy: 0.1042

  64/2400 [..............................] - ETA: 1:09 - loss: 7.0298 - pad_loss: 1.5273 - fr_loss: 5.5025 - pad_acc: 0.4531 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.1406 - fr_categorical_accuracy: 0.1406

...

2352/2400 [============================>.] - ETA: 0s - loss: 0.1097 - pad_loss: 0.0143 - fr_loss: 0.0954 - pad_acc: 0.9923 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.9894 - fr_categorical_accuracy: 0.9894

2368/2400 [============================>.] - ETA: 0s - loss: 0.1090 - pad_loss: 0.0142 - fr_loss: 0.0947 - pad_acc: 0.9924 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.9894 - fr_categorical_accuracy: 0.9894

2384/2400 [============================>.] - ETA: 0s - loss: 0.1082 - pad_loss: 0.0141 - fr_loss: 0.0941 - pad_acc: 0.9924 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.9895 - fr_categorical_accuracy: 0.9895

2400/2400 [==============================] - 24s 10ms/step - loss: 0.1075 - pad_loss: 0.0140 - fr_loss: 0.0935 - pad_acc: 0.9925 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.9896 - fr_categorical_accuracy: 0.9896


{'loss': [0.6048416107584247, 0.1075115014626014], 'pad_loss': [0.14576234838492533, 0.014023946299454575], 'fr_loss': [0.45907925684245127, 0.09348755493019174], 'pad_acc': [0.925, 0.9925], 'pad_categorical_accuracy': [1.0, 1.0], 'fr_acc': [0.9104166666666667, 0.9895833333333334], 'fr_categorical_accuracy': [0.9104166666666667, 0.9895833333333334]}


In [8]:
for layer in pad_model.layers:
    layer.trainable = True
pad_model.get_layer("fr_classif1").trainable = False
pad_model.get_layer("fr_classif2").trainable = False
pad_model.get_layer("fr_classif3").trainable = False
pad_model.get_layer("fr_classif4").trainable = False
pad_model.get_layer("fr_classif5").trainable = False
pad_model.get_layer("fr_classif").trainable = False
pad_model.get_layer("fr").trainable = False
pad_model.compile(optimizer='adam', loss=losses, metrics=['accuracy','categorical_accuracy'])

In [9]:
history = pad_model.fit(tr_imgs, {"pad":tr_labels, "fr":tr_ids}, batch_size=16, epochs=1)
print(history.history)

Epoch 1/1


  16/2400 [..............................] - ETA: 6:58 - loss: 0.1804 - pad_loss: 1.5487e-06 - fr_loss: 0.1804 - pad_acc: 1.0000 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.9375 - fr_categorical_accuracy: 0.9375

  32/2400 [..............................] - ETA: 3:54 - loss: 7.6419 - pad_loss: 0.5000 - fr_loss: 7.1419 - pad_acc: 0.8750 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.5312 - fr_categorical_accuracy: 0.5312    

  48/2400 [..............................] - ETA: 2:53 - loss: 9.7099 - pad_loss: 0.5833 - fr_loss: 9.1266 - pad_acc: 0.8542 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.4167 - fr_categorical_accuracy: 0.4167

  64/2400 [..............................] - ETA: 2:22 - loss: 11.4351 - pad_loss: 0.8125 - fr_loss: 10.6226 - pad_acc: 0.7969 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.3281 - fr_categorical_accuracy: 0.3281

...

2352/2400 [============================>.] - ETA: 1s - loss: 16.4493 - pad_loss: 1.4881 - fr_loss: 14.9612 - pad_acc: 0.6280 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.0714 - fr_categorical_accuracy: 0.0714

2368/2400 [============================>.] - ETA: 0s - loss: 16.4538 - pad_loss: 1.4848 - fr_loss: 14.9690 - pad_acc: 0.6288 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.0709 - fr_categorical_accuracy: 0.0709

2384/2400 [============================>.] - ETA: 0s - loss: 16.4514 - pad_loss: 1.4883 - fr_loss: 14.9632 - pad_acc: 0.6279 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.0713 - fr_categorical_accuracy: 0.0713

2400/2400 [==============================] - 57s 24ms/step - loss: 16.4391 - pad_loss: 1.4883 - fr_loss: 14.9507 - pad_acc: 0.6279 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.0721 - fr_categorical_accuracy: 0.0721


{'loss': [16.439069407582284], 'pad_loss': [1.4883333436581945], 'fr_loss': [14.95073610842228], 'pad_acc': [0.6279166666666667], 'pad_categorical_accuracy': [1.0], 'fr_acc': [0.07208333333333333], 'fr_categorical_accuracy': [0.07208333333333333]}


In [40]:
pad_model.save("padModel3_Replay_28122018_13epc.h5")

In [56]:
from keras.models import load_model
# model = load_model("MyVGGface.h5")
pad_model = load_model("padModel3_Replay_28122018_13epc.h5")

In [7]:
score = pad_model.evaluate(test_imgs,{"pad":test_labels,"fr":dummy_test_ids},batch_size=16 )
print(score)

  16/3200 [..............................] - ETA: 46s

  32/3200 [..............................] - ETA: 38s

  48/3200 [..............................] - ETA: 34s

  64/3200 [..............................] - ETA: 33s

  80/3200 [..............................] - ETA: 32s

...

3152/3200 [============================>.] - ETA: 0s

3168/3200 [============================>.] - ETA: 0s

3184/3200 [============================>.] - ETA: 0s

3200/3200 [==============================] - 30s 9ms/step


[71.05991859436035, 0.20579231282438257, 70.85412647247314, 0.90875, 1.0, 0.0709375, 0.0709375]


In [57]:
scores = pad_model.predict(test_imgs,batch_size=16,verbose=1)
print(scores[1].shape)
scores_pad = scores[0]
features = scores[1]


  16/3200 [..............................] - ETA: 1:27

  32/3200 [..............................] - ETA: 57s 

  48/3200 [..............................] - ETA: 48s

  64/3200 [..............................] - ETA: 43s

  80/3200 [..............................] - ETA: 39s

...

3152/3200 [============================>.] - ETA: 0s

3168/3200 [============================>.] - ETA: 0s

3184/3200 [============================>.] - ETA: 0s

3200/3200 [==============================] - 31s 10ms/step


(3200, 15)


In [58]:
np.savetxt("scores/scores_model3_13epc_Replay_28122018.txt",scores_pad)
# np.savetxt("scores/features_model3_Replay_28122018.txt",features)
# np.savetxt("scores/truelabels_model3_Replay_28122018.txt",test_labels) #TODO maybe save PAD and FR scores separatly

In [ ]:
print(scores)

[[-0.99516386]
 [-0.7635757 ]
 [-0.99999994]
 ...
 [ 0.99604744]
 [ 0.9993456 ]
 [ 0.99867606]]


In [ ]:
from keras.utils import plot_model
plot_model(pad_model, to_file='model.png',show_shapes=True)